# API Calls and Database Creation

### Attribution: [Data provided by Financial Modeling Prep](https://financialmodelingprep.com/developer/docs/), Yahoo! Finance

In [1]:
import yfinance as yf
from urllib.request import urlopen
import certifi
import json
from sqlalchemy import create_engine
import psycopg2
import numpy as np
import pandas as pd
import os

### Tickers

In [2]:
# Retrieve Current S&P Stock Tickers - as of 08/22/2024
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_tickers = pd.read_html(url)[0]["Symbol"].to_list()

# Handling Errors from Wiki
sp500_tickers.remove("BRK.B")
sp500_tickers.remove("BF.B")
sp500_tickers.extend(["BRK-B", "BF-B"])

### Yfinance API

In [4]:
# Download and format the stock data without multi-index, "Date" column is currently stored as a datetime obj

# Download using yfinance API
df = yf.download(sp500_tickers, start="2023-01-01", end="2024-08-22", interval="1d")

# Formatting
df.sort_index(axis=1, inplace=True)
df = df.stack(future_stack=True).reset_index()  # future_stack required due to future deprecation
df.columns.name = None

# Visual Inspection - instead of multi-index, each stock shown for each day, future transformations require groupby(["Ticker").transform()
print(df.shape)
df.head()

[*********************100%%**********************]  503 of 503 completed


(206733, 8)


,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2023-01-03,A,148.440598,150.039993,153.130005,148.470001,151.960007,1414300.0
1,2023-01-03,AAL,12.740000,12.740000,13.000000,12.530000,12.910000,21865100.0
2,2023-01-03,AAPL,123.904625,125.070000,130.899994,124.169998,130.279999,112117500.0
3,2023-01-03,ABBV,151.773148,162.380005,163.020004,160.809998,162.039993,4937500.0
4,2023-01-03,ABNB,84.900002,84.900002,88.139999,84.300003,87.385002,3901200.0


### Financial Modeling Prep

In [7]:
api_key = os.getenv('STOCK_API_KEY')
print(api_key)

None


In [8]:
from urllib.request import urlopen
import certifi
import json

def get_jsonparsed_data(url):
    """
    The cafile=certifi.where() part ensures that urlopen uses the certificate bundle provided by certifi 
    to verify the SSL certificate of the URL you're trying to access. Then read the api response and return it in json-form

    Args:
    ------------
    url:str | url with api key to successfully download information

    Returns:
    ------------
    json.loads(data):json | json object with the results of the query

    Raises:
    ------------
    HTTPError | if any part of the query string is inaccurate, users must have their own API key "HTTP Error 401: Unauthorized"

    Example : 
    url = ("https://financialmodelingprep.com/api/v3/search?query=AA&apikey=YOUR_API_KEY")
    get_jsonparsed_data(url)
    """
    response = urlopen(url, cafile=certifi.where())  
    data = response.read().decode("utf-8")  # read the API response
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/v3/search?query=AA&apikey=YOUR_API_KEY")
print(get_jsonparsed_data(url))